In [1]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns
import gget

In [2]:
geneNamesPath = "/scratch/indikar_root/indikar1/shared_data/experiment9/references/geneNames.csv"

gf = pd.read_csv(geneNamesPath)
print(f"{gf.shape=}")
print(gf['geneId'].nunique())

gf = gf[['geneId', 'geneName']].drop_duplicates()
print(f"{gf.shape=}")

gf.head()

gf.shape=(262041, 5)
41407
gf.shape=(41407, 2)


,geneId,geneName
0,ENSG00000160072,ATAD3B
7,ENSG00000225972,MTND1P23
9,ENSG00000224315,RPL7P7
11,ENSG00000198744,MTCO3P12
13,ENSG00000279928,DDX11L17


In [3]:
dirPath = "/scratch/indikar_root/indikar1/shared_data/experiment9/counts/"

df = gf.copy()

fileNames = []

for f in os.listdir(dirPath):
    fileId = f.split(".")[0]
    fullPath = f"{dirPath}{f}"
    tf = pd.read_csv(fullPath, header=None, sep="\t")

    tf.columns = ['geneId', fileId]
    fileNames.append(fileId)

    df = pd.merge(df, tf, 
                  how='left',
                  left_on='geneId',
                  right_on='geneId')
    
print(f"{df.shape=}")
df = df.fillna(0)
df.head()

df.shape=(41407, 3)


,geneId,geneName,merged
0,ENSG00000160072,ATAD3B,0
1,ENSG00000225972,MTND1P23,0
2,ENSG00000224315,RPL7P7,0
3,ENSG00000198744,MTCO3P12,0
4,ENSG00000279928,DDX11L17,0


In [4]:
df['merged'].describe()

count     41407.000000
mean         46.127321
std        1150.484983
min           0.000000
25%           0.000000
50%           0.000000
75%           0.000000
max      100025.000000
Name: merged, dtype: float64

In [5]:
df['binary'] = df['merged'].astype(bool).astype(int)
df['binary'].value_counts()

0    41086
1      321
Name: binary, dtype: int64

In [6]:
# drop non-expressed genes
exp = df[df['binary'] == 1].reset_index(drop=True)
exp = exp.sort_values(by='merged', ascending=False)
exp.head()

,geneId,geneName,merged,binary
266,ENSG00000132507,EIF5A,100025,1
121,ENSG00000075624,ACTB,75135,1
97,ENSG00000169045,HNRNPH1,65268,1
150,ENSG00000119335,SET,47178,1
196,ENSG00000111445,RFC5,38922,1


In [9]:
df = df.sort_values(by='merged', ascending=False)
df.head()

,geneId,geneName,merged,binary
34324,ENSG00000132507,EIF5A,100025,1
15875,ENSG00000075624,ACTB,75135,1
12531,ENSG00000169045,HNRNPH1,65268,1
21657,ENSG00000119335,SET,47178,1
26599,ENSG00000111445,RFC5,38922,1


In [8]:
# break

SyntaxError: 'break' outside loop (668683560.py, line 1)

In [13]:
top = 300
genes = df['geneName'].head(top).to_list()

db = 'PanglaoDB_Augmented_2021'
ef = gget.enrichr(genes, database=db)
pd.set_option('display.max_colwidth', 200)
ef[['path_name', 'p_val', 'overlapping_genes']].head(10)

Thu Jan 26 16:19:30 2023 INFO Performing Enichr analysis using database PanglaoDB_Augmented_2021.


,path_name,p_val,overlapping_genes
0,Pluripotent Stem Cells,0.001508,"[MSH6, SPN, NUP107, SET, NCL, FUBP1, NAP1L1]"
1,Chondrocytes,0.022975,"[CCDC80, C1R, NOG, CTNNB1, IGFBP7, DKK3]"
2,Osteocytes,0.060066,"[WWTR1, SERPINH1, SLIT2, CNN3]"
3,Fibroblasts,0.061200,"[CCDC80, C1R, SERPINH1, IGFBP7, VIM, DKK3, RUNX1]"
4,Airway Smooth Muscle Cells,0.061891,"[NOG, SERPINH1, IGFBP7, DKK3]"
5,Reticulocytes,0.069477,"[GLE1, HNRNPL, NUP107, POLR2B]"
6,Vascular Smooth Muscle Cells,0.069477,"[CCDC80, C1R, MYH11, SLIT2]"
7,Loop Of Henle Cells,0.070844,"[C1R, IGFBP7, TRPM7, THBS1, DKK3]"
8,Mesangial Cells,0.072469,"[CCDC80, C1R, ACTN1, CRIM1, DKK3]"
9,Peri-islet Schwann Cells,0.092655,"[MAP1B, SLIT2, DKK3, GULP1]"


In [ ]:
# t = 0
# genes = df[df['GeneCount'] > t]['geneName'].to_list()
# genes

In [ ]:
# t = 0
# genes = df[df['GeneCount'] > t]['geneName'].to_list()
# print(len(genes))

# db = 'PanglaoDB_Augmented_2021'
# ef = gget.enrichr(genes, database=db)
# pd.set_option('display.max_colwidth', 200)

# ef[['path_name', 'p_val', 'overlapping_genes']].head(10)

In [ ]:
# alpha = 0.05
# ef  = ef[ef['p_val'] <= alpha]
# ef.to_csv("CellTypePredictions.csv", index=False)
